# Laboratorul 3: Satisfacerea parțială a restricțiilor
 - Tudor Berariu
 - Andrei Olaru
 - Ana-Maria Simion

In [1]:
from copy import copy, deepcopy
from itertools import combinations
from __future__ import annotations

## O problemă de satisfacere a restricțiilor

O **problemă de satisfacere a restricțiilor** este definită de:
 - o mulțime discretă de **variabile**
 - câte un **domeniu de valori** pentru fiecare variabilă
 - un set de **constrângeri** impuse asupra unor grupuri de variabile
 
Vom reprezenta în Python cele de mai sus astfel:
 - fiecare variabilă va fi reprezentată printr-un șir de caractere

```
Vars = ["A", "B", "C"]
```
 - mulțimea domeniilor va fi un dicționar având câte o intrare pentru fiecare variabilă:
    + cheie va fi numele variabilei
    + valoare va fi domeniul de valori al acelei variabile

```
Domains = {"A": [1, 2, 3], "B": [1, 5, 9], "C": [-2, -1]}
```
 - o constrângere va fi reprezentată printr-un tuplu format din:
    + lista de variabile implicată în constrângere
    + o funcție anonimă care întoarce `True` sau `False`

```
Constraints = [(["A", "B", "C"], lambda a, b, c: a + b + c == 0)]
```

Vom reprezenta o **soluție** printr-un dicționar ce indică o valoare pentru fiecare variabilă (e.g. `{"A": 1, "B": 1, "C" -2}`) și vom defini **costul** ca fiind egal cu numărul de constrângeri încălcate de acea soluție.

In [2]:
VarsA = ["A", "B", "C", "D", "E"]
DomainsA = {v: [i for i in range(10)] for v in VarsA}
ConstraintsA = [(list(p), lambda x,y: x != y) for p in combinations(VarsA, 2)] # toate valorile diferite
ConstraintsA.append((["A","B"], lambda a, b: a + b == 10))
ConstraintsA.append((["B","D"], lambda b, d: b + d == 6))
ConstraintsA.append((["C"], lambda c: c < 5))
ConstraintsA.append((["A"], lambda a: a > 5))
ConstraintsA.append((["A","B","C","D","E"], lambda a, b, c, d, e: a + b + c + d + e == 30))
MathProblem = {"Vars": VarsA, "Domains": DomainsA, "Constraints": ConstraintsA}

In [3]:
VarsC = ["France", "Germany", "Loux", "Belgium", "Netherlands"]
DomainsC = {v: ["blue", "red", "yellow", "green"] for v in VarsC}
ConstraintsC = []
for (a, b) in [("France", "Germany"), ("France", "Belgium"), ("France", "Loux"),
               ("Belgium", "Netherlands"), ("Belgium", "Loux"), ("Belgium", "Germany"),
               ("Loux", "Germany"), ("Netherlands", "Germany")]:
    ConstraintsC.append(([a, b], lambda a, b: a != b))
ColoringProblem = {"Vars": VarsC, "Domains": DomainsC, "Constraints": ConstraintsC}

In [4]:
class State:
    # variantă redusă a clasei din laboratorul de căutare locală
    def __init__(
        self,
        size: int,
        board: list[int] | None = None,
        conflicts: int | None = None,
        seed: int = 42
        
    ) -> None:

        self.size = size
        self.board = board if board is not None else State.generate_board(size, seed)
        self.nconflicts = conflicts if conflicts is not None \
            else State.__compute_conflicts(self.size, self.board)

    @staticmethod
    def __compute_conflicts(size: int, board: list[int]) -> int:
        '''
        Calculează numărul de conflicte parcurgând toate perechile de dame
        '''
        _conflicts = 0
        for i in range(size):
            for j in range(i + 1, size):
                if board[i] == board[j]: _conflicts += 1
                if abs(board[i] - board[j]) == (j - i): _conflicts += 1

        return _conflicts

    def conflicts(self) -> int:
        '''
        Întoarce numărul de conflicte din această stare.
        '''
        return self.nconflicts

    def is_final(self) -> bool:
        '''
        Întoarce True dacă este stare finală.
        '''
        return self.nconflicts == 0

    def __str__(self) -> str:
        board = " " + "_ " * self.size + "\n"
        board += "|\n".join("|" + "|".join(("Q" if col == self.board[row] else"_")
                                            for row in range(self.size))
                                            for col in range(self.size))
        board += "|\n"
        return board

    def display(self) -> None:
        '''
        Afișează tablei de joc
        '''
        print(self)


In [5]:
VarsQ = ["Q" + str(i) for i in range(8)]
DomainsQ = {v: list(range(8)) for v in VarsQ}
ConstraintsQ = []
for q1 in range(7):
    for q2 in range(q1 + 1, 8):
        ConstraintsQ.append((["Q" + str(q1), "Q" + str(q2)], lambda q1_row, q2_row: q1_row != q2_row))
        ConstraintsQ.append((["Q" + str(q1), "Q" + str(q2)],
                                  lambda q1_row, q2_row, q1_col=q1, q2_col=q2:
                                  abs(q1_col - q2_col) != abs(q1_row - q2_row)))
EightQueensProblem = {"Vars": VarsQ, "Domains": DomainsQ, "Constraints": ConstraintsQ}

## Cerința 1

Implementați funcția `get_constraints` care primește o variabilă `var` și o listă de constrângeri `constraints` și întoarce doar acele constrângeri care implică variabila dată.
```
Constraints = [(["A", "B"], lambda a,b: a<b), (["A"], lambda a: a<5)]
get_constraints("B", Constraints)
==> [(["A", "B"], lambda a,b: a<b)]
get_constraints("A", Constraints)
==> [(["A", "B"], lambda a,b: a<b), (["A"], lambda a: a<5)]
```

In [6]:
def get_constraints(var, constraints):
    return [constraint for constraint in constraints if var in constraint[0]]

get_constraints("France", ConstraintsC) # => [(['France', 'Germany'], ...), (['France', 'Belgium'], ...), (['France', 'Loux'], ...)]

[(['France', 'Germany'], <function __main__.<lambda>(a, b)>),
 (['France', 'Belgium'], <function __main__.<lambda>(a, b)>),
 (['France', 'Loux'], <function __main__.<lambda>(a, b)>)]

## Cerința 2

Implementați funcția `fixed_constraints` care primește o soluție parțială `solution` și un set de constrângeri `constraints` și întoarce doar acele constrângeri care pot fi evaluate (i.e. variabilele implicate se regăsesc în soluția parțială).

In [7]:
def fixed_constraints(solution, constraints):
    return [constraint for constraint in constraints if all(var in solution for var in constraint[0])]

print(fixed_constraints({"France": "blue", "Belgium": "green"}, ConstraintsC)) # => [(['France', 'Belgium'], ...)]
print(fixed_constraints({"A": "1", "C": "2"}, ConstraintsA)) # => [(['A', 'C'], ...), (['C'], ...), (['A'], ...)]

[(['France', 'Belgium'], <function <lambda> at 0x000002C40C2162A0>)]
[(['A', 'C'], <function <lambda> at 0x000002C40C216660>), (['C'], <function <lambda> at 0x000002C40C216C00>), (['A'], <function <lambda> at 0x000002C40C216CA0>)]


## Cerința 3

Implementați funcția `check_constraint` care primește o constrângere `constraint` și o soluție parțială `solution` și întoarce `True` dacă soluția respectă constrângerea și `False` altfel.

**Hint:** pentru a apela o funcție știind lista parametrilor săi folosiți `f(*args)`, unde `args` este lista de argumente.

In [8]:
def check_constraint(solution, constraint):
    return constraint[1](*[solution[var] for var in constraint[0]])

print(check_constraint({"France": "blue", "Belgium": "green"}, ConstraintsC[1])) # => True
print(check_constraint({"France": "blue", "Belgium": "blue"}, ConstraintsC[1])) # => False
print(check_constraint({"C": 10, "A": 10}, ConstraintsA[-2])) # => True
print(check_constraint({"C": 10, "A": 3}, ConstraintsA[-2])) # => False

True
False
True
False


## Cerința 4: PCSP

Completați ceea ce lipsește în algoritmul PCSP.

* `vars` -- variabilele care mai rămân de verificat
* `domains` -- domeniile pentru variabilele care mai rămân de verificat, cu valorile care mai rămân de verificat pentru fiecare variabilă
* `constrains` -- lista de constrângeri
* `acceptable_cost` -- costul pentru care se consideră ca soluția este satisfăcătoare
* `solution` -- soluția parțială, conținând valori pentru variabilele verificate până acum
* `cost` -- costul soluției parțiale (`solution`) -- numărul de constrângeri nesatisfăcute

Valoare întoarsă de funcție este `True` dacă a fost găsită o soluție completă satisfăcătoare (vezi costul acceptabil), și `False` altfel.

Se folosesc două variabile globale:
* `best_cost` -- cel mai bun (mic) cost întâlnit până acum în explorare, pentru o soluție completă
* `best_solution` -- soluția corespunzătoare celui mai bun cost

In [9]:
def PCSP(vars, domains, constraints, acceptable_cost, solution, cost):
    global best_solution
    global best_cost
    global iterations
    if not vars:
        # Dacă nu mai sunt variabile, am ajuns la o soluție mai bună
        print("New best: " + str(cost) + " - " + str(solution))

        # Salvați soluția nou-descoperită
        best_solution = copy(solution)
        best_cost = cost

        # Dacă este suficient de bună, funcția întoarce True
        if best_cost <= acceptable_cost:
            return True
        
    elif not domains[vars[0]]:
        # Dacă nu mai sunt valori în domeniu, am terminat căutarea
        return False
    elif cost == best_cost:
        # Dacă am ajuns deja la un cost identic cu cel al celei mai bune soluții, nu mergem mai departe
        return False
    else:
        # Luăm prima variabilă și prima valoare din domeniu
        var = vars[0]
        val = domains[var].pop(0)
        iterations += 1

        # Construim noua soluție
        new_solution = copy(solution)
        new_solution[var] = val

        # Obținem lista constrângerilor ce pot fi evaluate acum
        eval_constraints = fixed_constraints(new_solution, constraints)

        # Calculăm costul noii soluții parțiale (fiecare constrângere încălcată = 1)
        new_cost = 0

        for constraint in eval_constraints:
            if not check_constraint(new_solution, constraint):
                new_cost += 1

        # Verificăm dacă noul cost este mai mic decât cel mai bun cost
        # După rezolvarea completă, introduceți și a doua condiție
        if new_cost < best_cost and new_cost <= acceptable_cost:

            # Dacă noul cost este mai mic decât cel mai bun cunoscut, rezolvăm pentru restul variabilelor
            # Dacă apelul recursiv întoarce True, a fost găsită o soluție suficient de bună, deci întoarcem True

            if PCSP(vars[1:], deepcopy(domains), constraints, acceptable_cost, new_solution, new_cost):
                return True
            
        # Verificăm pentru restul valorilor
        return PCSP(vars, deepcopy(domains), constraints, acceptable_cost, solution, cost)

# Un wrapper care să instanțieze variabilele globale
def run_pcsp(problem, acceptable_cost):
    global best_solution
    global best_cost
    global iterations

    [vars, domains, constraints] = [problem[e] for e in ["Vars", "Domains", "Constraints"]]

    best_solution = {}
    best_cost = len(constraints)
    iterations = 0

    if PCSP(vars, deepcopy(domains), constraints, acceptable_cost, {}, 0):
        print(f"Best found in {iterations} iterations: {str(best_cost)} - {str(best_solution)}")
    else:
        print(f"Acceptable solution not found in {iterations}; Best found: {str(best_cost)} - {str(best_solution)}")

# Rulăm testele
run_pcsp(MathProblem, 1)
run_pcsp(ColoringProblem, 1)
run_pcsp(EightQueensProblem, 1)
EightQueensSolution = list(best_solution.values())

New best: 1 - {'A': 6, 'B': 4, 'C': 0, 'D': 2, 'E': 1}
Best found in 588 iterations: 1 - {'A': 6, 'B': 4, 'C': 0, 'D': 2, 'E': 1}
New best: 1 - {'France': 'blue', 'Germany': 'blue', 'Loux': 'red', 'Belgium': 'yellow', 'Netherlands': 'red'}
Best found in 9 iterations: 1 - {'France': 'blue', 'Germany': 'blue', 'Loux': 'red', 'Belgium': 'yellow', 'Netherlands': 'red'}
New best: 1 - {'Q0': 0, 'Q1': 0, 'Q2': 3, 'Q3': 5, 'Q4': 7, 'Q5': 1, 'Q6': 4, 'Q7': 2}
Best found in 78 iterations: 1 - {'Q0': 0, 'Q1': 0, 'Q2': 3, 'Q3': 5, 'Q4': 7, 'Q5': 1, 'Q6': 4, 'Q7': 2}


Expected for numbers:

```
New best: 14 - {'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 0}
New best: 10 - {'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 1}
New best: 8 - {'A': 0, 'B': 0, 'C': 0, 'D': 1, 'E': 1}
New best: 7 - {'A': 0, 'B': 0, 'C': 0, 'D': 1, 'E': 2}
New best: 6 - {'A': 0, 'B': 0, 'C': 0, 'D': 6, 'E': 1}
New best: 5 - {'A': 0, 'B': 0, 'C': 1, 'D': 2, 'E': 3}
New best: 4 - {'A': 0, 'B': 0, 'C': 1, 'D': 6, 'E': 2}
New best: 3 - {'A': 0, 'B': 1, 'C': 2, 'D': 5, 'E': 3}
New best: 2 - {'A': 2, 'B': 8, 'C': 4, 'D': 7, 'E': 9}
New best: 1 - {'A': 6, 'B': 4, 'C': 0, 'D': 2, 'E': 1}
Best found: 1 - {'A': 6, 'B': 4, 'C': 0, 'D': 2, 'E': 1}
```

Expected for country colors:

```
New best:  8  -  {'Loux': 'blue', 'Belgium': 'blue', 'Netherlands': 'blue', 'Germany': 'blue', 'France': 'blue'}
New best:  6  -  {'Loux': 'blue', 'Belgium': 'blue', 'Netherlands': 'red', 'Germany': 'blue', 'France': 'blue'}
New best:  4  -  {'Loux': 'blue', 'Belgium': 'red', 'Netherlands': 'blue', 'Germany': 'blue', 'France': 'blue'}
New best:  3  -  {'Loux': 'blue', 'Belgium': 'red', 'Netherlands': 'yellow', 'Germany': 'blue', 'France': 'blue'}
New best:  2  -  {'Loux': 'red', 'Belgium': 'red', 'Netherlands': 'yellow', 'Germany': 'blue', 'France': 'blue'}
New best:  1  -  {'Loux': 'red', 'Belgium': 'yellow', 'Netherlands': 'red', 'Germany': 'blue', 'France': 'blue'}
Best found: 1  -  {'Loux': 'red', 'Belgium': 'yellow', 'Netherlands': 'red', 'Germany': 'blue', 'France': 'blue'}
```

Expected for 8-Queens:

```
New best: 28 - {'Q0': 0, 'Q1': 0, 'Q2': 0, 'Q3': 0, 'Q4': 0, 'Q5': 0, 'Q6': 0, 'Q7': 0}
New best: 22 - {'Q0': 0, 'Q1': 0, 'Q2': 0, 'Q3': 0, 'Q4': 0, 'Q5': 0, 'Q6': 0, 'Q7': 1}
New best: 17 - {'Q0': 0, 'Q1': 0, 'Q2': 0, 'Q3': 0, 'Q4': 0, 'Q5': 0, 'Q6': 1, 'Q7': 1}
New best: 16 - {'Q0': 0, 'Q1': 0, 'Q2': 0, 'Q3': 0, 'Q4': 0, 'Q5': 0, 'Q6': 3, 'Q7': 1}
New best: 15 - {'Q0': 0, 'Q1': 0, 'Q2': 0, 'Q3': 0, 'Q4': 0, 'Q5': 0, 'Q6': 7, 'Q7': 1}
New best: 14 - {'Q0': 0, 'Q1': 0, 'Q2': 0, 'Q3': 0, 'Q4': 0, 'Q5': 1, 'Q6': 1, 'Q7': 1}
New best: 13 - {'Q0': 0, 'Q1': 0, 'Q2': 0, 'Q3': 0, 'Q4': 0, 'Q5': 1, 'Q6': 1, 'Q7': 2}
New best: 12 - {'Q0': 0, 'Q1': 0, 'Q2': 0, 'Q3': 0, 'Q4': 0, 'Q5': 1, 'Q6': 4, 'Q7': 2}
New best: 11 - {'Q0': 0, 'Q1': 0, 'Q2': 0, 'Q3': 0, 'Q4': 0, 'Q5': 1, 'Q6': 7, 'Q7': 2}
New best: 10 - {'Q0': 0, 'Q1': 0, 'Q2': 0, 'Q3': 0, 'Q4': 1, 'Q5': 1, 'Q6': 2, 'Q7': 2}
New best: 9 - {'Q0': 0, 'Q1': 0, 'Q2': 0, 'Q3': 0, 'Q4': 1, 'Q5': 1, 'Q6': 4, 'Q7': 2}
New best: 8 - {'Q0': 0, 'Q1': 0, 'Q2': 0, 'Q3': 0, 'Q4': 1, 'Q5': 1, 'Q6': 7, 'Q7': 2}
New best: 7 - {'Q0': 0, 'Q1': 0, 'Q2': 0, 'Q3': 0, 'Q4': 2, 'Q5': 6, 'Q6': 1, 'Q7': 3}
New best: 6 - {'Q0': 0, 'Q1': 0, 'Q2': 0, 'Q3': 0, 'Q4': 5, 'Q5': 7, 'Q6': 1, 'Q7': 3}
New best: 5 - {'Q0': 0, 'Q1': 0, 'Q2': 0, 'Q3': 1, 'Q4': 3, 'Q5': 7, 'Q6': 2, 'Q7': 4}
New best: 4 - {'Q0': 0, 'Q1': 0, 'Q2': 0, 'Q3': 1, 'Q4': 5, 'Q5': 7, 'Q6': 2, 'Q7': 4}
New best: 3 - {'Q0': 0, 'Q1': 0, 'Q2': 1, 'Q3': 1, 'Q4': 5, 'Q5': 7, 'Q6': 2, 'Q7': 4}
New best: 2 - {'Q0': 0, 'Q1': 0, 'Q2': 2, 'Q3': 4, 'Q4': 6, 'Q5': 1, 'Q6': 3, 'Q7': 5}
New best: 1 - {'Q0': 0, 'Q1': 0, 'Q2': 3, 'Q3': 5, 'Q4': 7, 'Q5': 1, 'Q6': 4, 'Q7': 2}
Best found: 1 - {'Q0': 0, 'Q1': 0, 'Q2': 3, 'Q3': 5, 'Q4': 7, 'Q5': 1, 'Q6': 4, 'Q7': 2}
```

In [10]:
if EightQueensSolution:
    state = State(8, EightQueensSolution)
    state.display()
    print(f"Number of conflicts: {state.conflicts()}")
    print(f"Is{'' if state.is_final() else ' not'} solution.\n")

 _ _ _ _ _ _ _ _ 
|Q|Q|_|_|_|_|_|_|
|_|_|_|_|_|Q|_|_|
|_|_|_|_|_|_|_|Q|
|_|_|Q|_|_|_|_|_|
|_|_|_|_|_|_|Q|_|
|_|_|_|Q|_|_|_|_|
|_|_|_|_|_|_|_|_|
|_|_|_|_|Q|_|_|_|

Number of conflicts: 1
Is not solution.

